In [1]:
#import pandas and numpy libraries for data audit and cleaning
import pandas as pd
import numpy as np

#Load training and test data from the provided datasets

Data_train = pd.read_csv("train.csv",header=0)
Data_test = pd.read_csv("test.csv",header=0)

#copy data for playing
trainData=Data_train.copy(deep=True)
testData=Data_test.copy(deep=True)

#cleaning the data for missing values
trainData['Embarked']=trainData['Embarked'].fillna(value='S')

#Replace missing age values with average value for that gender
condition=[(trainData['Age'].notnull()),((trainData['Sex'] == 'male')&(trainData['Age'].isnull())),((trainData['Sex'] == 'female')&(trainData['Age'].isnull()))]
values=[trainData['Age'],trainData.query("Sex == 'male'").agg({'Age':np.mean}),trainData.query("Sex == 'female'").agg({'Age':np.mean})]
trainData['Age']=np.select(condition,values)

condition=[(testData['Age'].notnull()),((testData['Sex'] == 'male')&(testData['Age'].isnull())),((testData['Sex'] == 'female')&(testData['Age'].isnull()))]
values=[testData['Age'],testData.query("Sex == 'male'").agg({'Age':np.mean}),testData.query("Sex == 'female'").agg({'Age':np.mean})]
testData['Age']=np.select(condition,values)

#create training and test datasets
X_train=trainData.drop(['PassengerId','Survived','Name','Ticket','Cabin'],axis=1)
Y_train=trainData['Survived']
X_test=testData.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

X_train=pd.get_dummies(data=X_train)
X_test=pd.get_dummies(data=X_test)

X_test['Fare']=X_test['Fare'].fillna(value=np.mean(X_test['Fare']))

In [2]:
#Import Gradient Boost
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
#Build model
clf=GradientBoostingClassifier(n_estimators=50,random_state=400)

In [4]:
#fit the model
clf.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=50, presort='auto', random_state=400,
              subsample=1.0, verbose=0, warm_start=False)

In [6]:
#read the sample output file to get the estimate accuracy
Y_test=pd.read_csv("gender_submission.csv",header=0)
Y_test=Y_test.drop(['PassengerId'],axis=1)

In [7]:
#calculate accuracy for the model
clf.score(X_test,Y_test)

0.90909090909090906

In [8]:
#Grid Search CV to get best number of trees for Gradient Boosting
from sklearn.model_selection import GridSearchCV

In [9]:
est=list(range(10,300,10))
mod=GridSearchCV(clf,param_grid={'n_estimators':est})
mod.fit(X_train,Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=50, presort='auto', random_state=400,
              subsample=1.0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [10]:
#get the best estimator value and create the model with it as a parameter
mod.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=400,
              subsample=1.0, verbose=0, warm_start=False)

In [25]:
clf=GradientBoostingClassifier(n_estimators=20,random_state=400)

In [26]:
# Fit the new model and get accuracy
clf.fit(X_train,Y_train)
clf.score(X_test,Y_test)

0.91148325358851678

In [27]:
# get predictions and write them in a csv for submission
pred=clf.predict(X_test)

In [29]:
op=pd.DataFrame(testData['PassengerId'],columns=['PassengerId'])
op['Survived']=pd.Series(pred)

In [30]:
op.to_csv('GradBoost.csv',header=['PassengerId','Survived'],index=False)